[Link](https://www.kaggle.com/competitions/digit-recognizer/data) to Competition

I went through this to get a better understanding of the concepts I've learned so far in fastai. Also to brush up on basic numpy, pandas, etc. I used Ahed Jneed's great notebook [here](https://www.kaggle.com/code/ahedjneed/mnist-with-fastai/notebook) as a guidepost whenever I got stuck.

In [1]:
#|default_exp app

In [2]:
#| export
import os
from pathlib import Path
import pandas as pd
import numpy as np
from fastai.vision.all import *
from fastai.vision.all import Image
from torchvision.utils import save_image
from fastai.callback.all import *

matplotlib.rc('image', cmap='Greys')

In [3]:
#| export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
creds = ''

In [4]:
#| export
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [5]:
#| export
path = Path('digit-recognizer')

In [6]:
#| export
if not iskaggle and not path.exists():
    import zipfile, kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [7]:
#| export
if iskaggle:
    path = Path('../input/digit-recognizer')
    ! pip install -q dataset

Import Training Set

In [8]:
#| export
train_full = pd.read_csv(path/'train.csv')
test_full = pd.read_csv(path/'test.csv')

In [9]:
test_full.shape

(28000, 784)

In [10]:
#| export
label = train_full.iloc[:, :1]

In [11]:
#| export
image_df = train_full.iloc[:, 1:]
test_image_df = train_full.iloc[:, 1:]

In [12]:
image_df.shape, test_image_df.shape

((42000, 784), (28000, 783))

In [13]:
#| export
np_image_array = image_df.values
np_test_array = test_image_df.values

In [14]:
#| export
image_tens = [tensor(x).float() for x in np_image_array]
test_image_tens = [tensor(x).float() for x in np_test_array]

In [15]:
#| export
stacked_img_tens = torch.stack(image_tens).float()/255
stacked_test_tens = torch.stack(test_image_tens).float()/255

In [16]:
#| export
stacked_label_tens = tensor(label).unsqueeze(1)
stacked_test_label_tens = tensor(label).unsqueeze(1)

In [17]:
# img = stacked_img_tens[0].reshape(28, 28)
# save_image(img, './test.png')

In [18]:
#| export
train_path = Path('train')
test_path = Path('test')
def tens_to_img(folder_path, stacked_tensor, labeled=True):
    if not folder_path.exists():
        folder_path.mkdir()
    for i in range(stacked_tensor.shape[0]):
        img = stacked_tensor[i].reshape(28, 28)
        if labeled:
            dest = (folder_path/label['label'][i].astype(str))
        else:
            dest = (folder_path)
        dest.mkdir(exist_ok=True)
        img_path = Path(f"{dest}/{i}.png")
        if not img_path.exists():
            save_image(img, img_path)

In [19]:
#| export
tens_to_img(train_path, stacked_img_tens)
tens_to_img(test_path, stacked_test_tens, labeled=False)

RuntimeError: shape '[28, 28]' is invalid for input of size 783

In [ ]:
# test_path = Path('test')
# if not folder_path.exists():
#     folder_path.mkdir()
# for i in range(stacked_test_tens.shape[0]):
#     img = stacked_test_tens[i].reshape(28, 28)
#     dest = (folder_path/label['label'][i].astype(str))
#     dest.mkdir(exist_ok=True)
#     img_path = Path(f"{dest}/{i}.png")
#     if not img_path.exists():
#         save_image(img, img_path)

In [ ]:
#| export
dls = ImageDataLoaders.from_folder(train_path, get_image_files(train_path), bs=255, valid_pct=0.2, seed=42, label_func=parent_label)

In [ ]:
#| export
learn = vision_learner(dls, resnet18, loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
#| export
learn.fine_tune(3, base_lr=1e-3)

In [ ]:
#| export
def find_lr_metrics(learn):
    lr_min, lr_steep, lr_valley = learn.lr_find(suggest_funcs=(minimum, steep, valley))
    lr_min = f"{lr_min:.2e}"
    lr_steep = f"{lr_steep:.2e}"
    lr_valley = f"{lr_valley:.2e}"
    print(f"Minimum: {lr_min}, Steepest Point {lr_steep}, Valley {lr_valley}")
    return lr_min, lr_steep, lr_valley

In [ ]:
find_lr_metrics(learn)

In [ ]:
#| export
learn = vision_learner(dls, resnet18, loss_func=F.cross_entropy, metrics=accuracy).to_fp16()
learn.fit_one_cycle(3, 1e-3)

In [ ]:
find_lr_metrics(learn)

In [ ]:
#| export
learn.unfreeze()

In [ ]:
find_lr_metrics(learn)

In [ ]:
#| export
learn.fit_one_cycle(5, lr_max=slice(1e-3, 1e-2))

In [ ]:
learn.recorder.plot_loss()

In [ ]:
#| export
learn.save('model_1')

In [ ]:
#| export
path = Path('test')
test_image_files = get_image_files(path)
test_dl = dls.test_dl(test_image_files)

In [ ]:
test_dl.show_batch()

In [ ]:
preds, _, pred_labels = learn.get_preds(dl=test_dl, with_decoded=True)

In [ ]:
preds.shape, pred_labels.shape

In [ ]:
probs = preds[0].tolist()
probs_df = pd.DataFrame({'Image Type': list(range(len(probs))), 'Probability': probs})
probs_df

In [ ]:
preds = np.argmax(preds, axis=1)

In [ ]:
preds[9].item()

In [ ]:
path = Path('digit-recognizer')

In [ ]:
df = pd.read_csv(path/'sample.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
len(df)

In [ ]:
test_path.ls()

In [ ]:
prediction_list = [preds[i].item() for i in  ]

In [ ]:
import nbdev
nbdev.export.nb_export('mnist_classifier.ipynb', 'app')
print('Export successful')